In [119]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

maybe get also likes and uses...

In [122]:
huggingface_languages_link = 'https://huggingface.co/languages'

In [121]:
def extract_huggingface_languages(url):
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'html.parser')
    # Initialize lists to store extracted data
    languages = []
    iso_codes = []
    datasets_links = []
    models_links = []
    datasets_numbers = []
    model_numbers = []

    # Extract data from each row of the table
    for row in soup.find_all('tr'):
        cols = row.find_all('td')
        if len(cols) > 0:  # Ensure it's not the header row
            language = cols[0].get_text(strip=True)
            iso_code = cols[1].find('code').get_text(strip=True)
            datasets_link = "https://huggingface.co" + cols[2].find('a')['href']
            models_link = "https://huggingface.co" + cols[3].find('a')['href']
            datasets_number = int(cols[2].find('a').get_text(strip=True).replace(',', ''))
            model_number = int(cols[3].find('a').get_text(strip=True).replace(',', ''))
            
            # Append data to lists
            languages.append(language)
            iso_codes.append(iso_code)
            datasets_links.append(datasets_link)
            models_links.append(models_link)
            datasets_numbers.append(datasets_number)
            model_numbers.append(model_number)

    # Create DataFrame
    df = pd.DataFrame({
        'Language': languages,
        'ISO Code': iso_codes,
        'Datasets Link': datasets_links,
        'Models Link': models_links,
        'Datasets Number': datasets_numbers,
        'Model Number': model_numbers
    })
    return df

In [123]:
df = extract_huggingface_languages(huggingface_languages_link)

In [124]:
df

,Language,ISO Code,Datasets Link,Models Link,Datasets Number,Model Number
0,EnglishEnglish,en,https://huggingface.co/datasets?language=langu...,https://huggingface.co/models?language=en,7792,34087
1,Chinese中文,zh,https://huggingface.co/datasets?language=langu...,https://huggingface.co/models?language=zh,922,3495
2,FrenchFrançais,fr,https://huggingface.co/datasets?language=langu...,https://huggingface.co/models?language=fr,823,3237
3,SpanishEspañol,es,https://huggingface.co/datasets?language=langu...,https://huggingface.co/models?language=es,646,2523
4,RussianРусский,ru,https://huggingface.co/datasets?language=langu...,https://huggingface.co/models?language=ru,620,1752
...,...,...,...,...,...,...
2041,Peruvian Sign Language,prl,https://huggingface.co/datasets?language=langu...,https://huggingface.co/models?language=prl,1,3
2042,Venezuelan Sign Language,vsl,https://huggingface.co/datasets?language=langu...,https://huggingface.co/models?language=vsl,1,2
2043,Sabah Malay,msi,https://huggingface.co/datasets?language=langu...,https://huggingface.co/models?language=msi,1,14
2044,Mexican Sign Language,mfs,https://huggingface.co/datasets?language=langu...,https://huggingface.co/models?language=mfs,1,3


In [139]:
def process_huggingface_datasets(dataset_url):
    dataset_request = requests.get(dataset_url).text
    dataset_soup = BeautifulSoup(dataset_request, 'html.parser')
    # Extract text and href attributes for all articles
    articles_data = []
    articles = dataset_soup.find_all('article')
    for article in articles:
        text = article.h4.text.strip()
        href = article.a['href']
        div = article.find('div').get_text(strip=True)
        #this part does not work very well yet...(the downloads and likes one)
        likes = re.findall(r'\b\d+\b', div)[0]
        downloads = re.findall(r'\b\d+\b', div)[1]
        articles_data.append({'dataset': text, 'dataset_url': href, 'downloads':downloads, 'likes':likes})
        

    # Convert to DataFrame
    df = pd.DataFrame(articles_data)
    return df 

In [140]:
dataset_stats = process_huggingface_datasets('https://huggingface.co/datasets?language=language:ofs')

In [141]:
dataset_stats

,dataset,dataset_url,downloads,likes
0,tatoeba,/datasets/tatoeba,967,18
1,lbourdois/language_tags,/datasets/lbourdois/language_tags,2,7
2,lbourdois/panlex,/datasets/lbourdois/panlex,1,6


In [142]:
longer_dataset_stats = process_huggingface_datasets('https://huggingface.co/datasets?language=language:or')

In [143]:
longer_dataset_stats

,dataset,dataset_url,downloads,likes
0,wikimedia/wikipedia,/datasets/wikimedia/wikipedia,332,9
1,opus100,/datasets/opus100,75,13
2,wikipedia,/datasets/wikipedia,398,18
3,wikimedia/wikisource,/datasets/wikimedia/wikisource,2023,8
4,facebook/belebele,/datasets/facebook/belebele,2023,15
5,mozilla-foundation/common_voice_16_0,/datasets/mozilla-foundation/common_voice_16_0,2023,21
6,cc100,/datasets/cc100,4,18
7,common_voice,/datasets/common_voice,120,5
8,oscar,/datasets/oscar,17,18
9,qed_amara,/datasets/qed_amara,220,18


In [106]:
def extract_next_page_url(soup):
    sections = soup.find_all('section')[-1]
    if sections:
        next_page_link = last_section.find('nav').find_all('a')[-1].get('href')
    if next_page_link:
        return next_page_link
    else:
        print('no href found')

In [107]:
trial = extract_next_page_url(soup)

In [108]:
trial

'?language=language:or&p=1&sort=trending'

In [115]:
def find_data_per_language(language_url):
    language_file = requests.get(language_url)
    if language_file.status_code == 200:
        soup = BeautifulSoup(language_file.text, 'html.parser')
        next_page_url = extract_next_page_url(soup) 
        while next_page_url:
            next_file = requests.get("https://huggingface.co" + next_page_url)
            if language_file.status_code == 200:
                new_soup = BeautifulSoup(language_file.text, 'html.parser')
                return new_soup
            else:
                print('Error: Unable to fetch language URL:', language_url)
                break

In [105]:
def extract_datasets(soup):
    articles_data = []
    articles = soup.find_all('article')
    for article in articles:
        text = article.h4.text.strip()
        href = article.a['href']
        div = article.find('div').get_text(strip=True)
        likes = re.findall(r'\b\d+k?\b', div)[-2]
        downloads = re.findall(r'\b\d+k?\b', div)[-1]
        articles_data.append({'dataset': text, 'dataset_url': href, 'downloads':downloads, 'likes':likes})
    
    df = pd.DataFrame(articles_data)
    return df 

In [111]:
extract_datasets(BeautifulSoup(trial, 'html.parser'))

""


In [117]:
tr = extract_datasets(trial)

In [118]:
tr

,dataset,dataset_url,downloads,likes
0,tatoeba,/datasets/tatoeba,23,967
1,lbourdois/language_tags,/datasets/lbourdois/language_tags,2,7
2,lbourdois/panlex,/datasets/lbourdois/panlex,1,6


In [116]:
trial = find_data_per_language('https://huggingface.co/datasets?language=language:ofs')